In [1]:
require 'daru'
require 'daru_plotting_plotly'
require 'rbplotly'
require 'hg'

Daru.plotting_library = :plotly

class Numeric
  def mili
    (self * 10 ** (-3)).to_f
  end
  
  def micro
    (self * 10 ** (-6)).to_f
  end
end

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

:micro

In [2]:
def map_color(x)
  if x == 'yellow'
    'orange'
  elsif x == 'white'
    'black'
  else
    x
  end
end

['red.csv', 'yellow.csv', 'blue.csv', 'white.csv', 'green.csv'].map { |name| [name, Daru::DataFrame.from_csv(name)] }.map do |name, df|
  color = File.basename(name, '.csv')
  {
    name: color,
    x: df['v'].to_a,
    y: df['i'].to_a,
    type: :scatter,
    marker: { color: map_color(color) }
  }
end.tap do |data|
  Plotly::Plot.new(data: data).show
end
nil

#<Plotly::Offline::HTML:0x007fb544b6c780 @id="b61c657a-67e6-4912-969d-8d52081408ee", @data=[{:name=>"red", :x=>[0.54, 1.076, 1.629, 1.72, 1.78, 1.863, 1.934, 1.996], :y=>[1.0e-14, 2.0e-14, 6.0e-06, 0.00068, 0.0023, 0.006, 0.011, 0.0146], :type=>:scatter, :marker=>{:color=>"red"}}, {:name=>"yellow", :x=>[1.62, 1.768, 1.817, 1.897, 1.945, 1.983, 2.01], :y=>[2.8e-05, 0.000505, 0.0013000000000000002, 0.004900000000000001, 0.0087, 0.012400000000000001, 0.0162], :type=>:scatter, :marker=>{:color=>"orange"}}, {:name=>"blue", :x=>[4.01, 3.54, 3.22, 3.03, 2.72, 2.32, 1.64, 1.18], :y=>[0.0146, 0.0078, 0.0038, 0.002, 0.00038, 2.8e-05, 1.1e-05, 5.7999999999999995e-06], :type=>:scatter, :marker=>{:color=>"blue"}}, {:name=>"white", :x=>[3.23, 3.03, 2.83, 2.74, 2.61, 2.5, 2.33, 0.852], :y=>[0.0154, 0.0075, 0.0026000000000000003, 0.0013000000000000002, 0.00038, 6.0e-05, 1.0e-06, 8.0e-08], :type=>:scatter, :marker=>{:color=>"black"}}, {:name=>"green", :x=>[3.11, 2.96, 2.82, 2.73, 2.59, 2.04, 1.5], :y=>[0.0156, 0.008400000000000001, 0.0034700000000000004, 0.00161, 0.000388, 1.0e-06, 1.0e-07], :type=>:scatter, :marker=>{:color=>"green"}}], @layout={}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>